In [1]:
import requests
import json
import arrow
import os
import hashlib
import sys
from ics import Calendar, Event
from ics.contentline.container import Container, ContentLine
from ics.types import ContainerItem
from datetime import datetime

In [2]:
# Static Values
headers = {
    'x-api-key': "0TvQnueqKa5mxJntVWt0w4LpLfEkrV1Ta8rQBb9Z" # public api key! This key does not have to be kept private
}
output_dir = "./leagues/"

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)


In [3]:
def generate_empty_calender(name):
    c = Calendar(creator="ics.tools Combinder v1.0")
    c.extra.append(ContentLine(name='NAME', value=name))
    c.extra.append(ContentLine(name='X-WR-CALNAME', value=name))
    c.extra.append(ContentLine(name='METHOD', value="PUBLISH"))
    return c

In [4]:
def get_leagues():
    r=requests.get("https://esports-api.lolesports.com/persisted/gw/getLeagues", headers=headers, params={"hl": "de-DE"})
    if r.status_code == 200:
        return json.loads(r.content)['data']['leagues']
    else:
        print("No leagues available")
        print(r)
        sys.exit(1)

In [5]:
def get_schedule(league_ids, pageToken):
    r=requests.get("https://esports-api.lolesports.com/persisted/gw/getSchedule", headers=headers, params={
        "hl": "de-DE",
        "leagueId": ",".join(league_ids),
        "pageToken": pageToken
    })
    if r.status_code == 200:
        content = json.loads(r.content)['data']['schedule']
        return content['pages']['older'], content['events']
    else:
        print("No schedules available")
        print(r)
        sys.exit(1)

In [6]:
def get_schedule_all_pages(league_ids):
    events = []
    token = None
    while True:
        token, evts = get_schedule(league_ids, token)
        print("Got page with {} events".format(len(evts)))
        if evts == None:
            break
        for e in evts:
            events.append(e)
        if token == None:
            break
    return events

In [7]:
def get_description(l_event):
    #Liga: WM
    #Phase: Gruppenphase
    #Teams: Blub vs. Nano
    #Strategy: bestOf 5
    #Status: 
    #Ergebnisse: Blub (5) : Nano (1)
    desc = f"Liga: {l_event['league']['name']}\n"
    desc += f"Phase: {l_event['blockName']}\n"
    desc += f"Teams: {l_event['match']['teams'][0]['name']} vs. {l_event['match']['teams'][1]['name']}\n"
    desc += f"Spiel-Strategie: {l_event['match']['strategy']['type']} {l_event['match']['strategy']['count']}\n"
    desc += f"Status: {l_event['state']}\n"
    if l_event['state'] == "completed":
        desc += f"Ergebnisse: {l_event['match']['teams'][0]['name']} ({l_event['match']['teams'][0]['result']['gameWins']}) vs. {l_event['match']['teams'][1]['name']} ({l_event['match']['teams'][1]['result']['gameWins']})\n"
    return desc

In [8]:
leagues = get_leagues()
league_ids = list(map(lambda d: d['id'], leagues))
league_calendar = {}

for league in leagues:
    if os.path.isfile(output_dir + league['slug'] + ".ics"):
        with open(output_dir + league['slug'] + ".ics", "r", encoding="utf-8") as file1:
            l = file1.read()
        league_calendar[league['slug']] = Calendar(l)
        print("Old League found", league['slug'])
    else:
        league_calendar[league['slug']] = generate_empty_calender(f"{league['name']} Spielplan")
        print("New League found", league['slug'])

Old League found european-masters
Old League found turkey-academy-league
Old League found lla
Old League found pcs
Old League found superliga
Old League found ultraliga
Old League found primeleague
Old League found pg_nationals
Old League found nlc
Old League found liga_portuguesa
Old League found lfl
Old League found hitpoint_masters
Old League found greek_legends
Old League found esports_balkan_league
Old League found cblol_academy
Old League found lco
Old League found ljl_academy
Old League found vcs
Old League found elite_series
Old League found honor_division
Old League found elements_league
Old League found volcano_discover_league
Old League found claro_gaming_stars_league
New League found master_flow_league
Old League found entel_honor_leagu
Old League found movistar_fiber_golden_league
Old League found worlds
Old League found all-star
Old League found lcs
Old League found lec
Old League found lck
Old League found lpl
Old League found msi
Old League found cblol-brazil
Old League

In [9]:
events = get_schedule_all_pages(league_ids)
events.sort(key=lambda e: e['match']['id'])

Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80 events
Got page with 80

In [10]:
len(events)

10156

In [11]:
for l_event in events:
    # Skip current running games
    if l_event['state'] == "inProgress" or not "match" in l_event:
        continue
    calendar = league_calendar[l_event['league']['slug']]
    uid = l_event['league']['slug'] + "-" + l_event['match']['id'] + "@lol.ics.tools"
    event = None
    is_new = True
    # Suche nach Kalender Event für Match Id
    for tevent in calendar.events:
        if tevent.uid == uid:
            event = tevent
            is_new = False
            break
    
    if is_new:
        print("New event for " + uid)
        event = Event()
        event.uid = uid
        event.created = datetime.now()
        
    event_name = f"[{l_event['league']['name']}] {l_event['match']['teams'][0]['code']} vs. {l_event['match']['teams'][1]['code']}"
    event_begin = arrow.get(l_event['startTime'])
    event_end = event_begin.shift(hours=l_event['match']['strategy']['count'])
    event_description = get_description(l_event)
    changed = False

    print(event_name)

    if event_name != event.summary:
        changed = True
        print("Old Name: {}, New Name: {}".format(event.summary, event_name))
        event.summary = event_name

    # Check if new start is after end
    if event.end is not None and event_begin.datetime > event.end:
        print("Special case: New start is after old end!")
        if event_end != event.end:
            changed = True
            print("Old End {}, New End {}".format(event.end, event_end.datetime))
            event.end = event_end.datetime

        if event_begin != event.begin:
            changed = True
            print("Old Start {}, New Start {}".format(event.begin, event_begin.datetime))
            event.begin = event_begin.datetime
    else:
        if event_begin != event.begin:
            changed = True
            print("Old Start {}, New Start {}".format(event.begin, event_begin.datetime))
            event.begin = event_begin.datetime
    
        if event_end != event.end:
            changed = True
            print("Old End {}, New End {}".format(event.end, event_end.datetime))
            event.end = event_end.datetime
    
    if event_description != event.description:
        changed = True
        print("Description changed")
        event.description = event_description
    
    if changed:
        event.last_modified = arrow.now().datetime
        
    if is_new:
        calendar.events.append(event)

[Liga Latinoamérica] INF vs. AK
[Liga Latinoamérica] FG vs. PIX
[Liga Latinoamérica] KLG vs. XTN
[Liga Latinoamérica] XTN vs. PIX
[Liga Latinoamérica] R7 vs. AK
[Liga Latinoamérica] ISG vs. XTN
[Liga Latinoamérica] INF vs. FG
[Liga Latinoamérica] ISG vs. KLG
[Liga Latinoamérica] R7 vs. INF
[Liga Latinoamérica] INF vs. FG
[Liga Latinoamérica] R7 vs. ISG
[Liga Latinoamérica] KLG vs. PIX
[Liga Latinoamérica] INF vs. XTN
[Liga Latinoamérica] R7 vs. FG
[Liga Latinoamérica] XTN vs. KLG
[Liga Latinoamérica] R7 vs. PIX
[Liga Latinoamérica] ISG vs. FG
[Liga Latinoamérica] R7 vs. KLG
[Liga Latinoamérica] ISG vs. AK
[Liga Latinoamérica] PIX vs. R7
[Liga Latinoamérica] FG vs. AK
[Liga Latinoamérica] KLG vs. AK
[Liga Latinoamérica] FG vs. PIX
[Liga Latinoamérica] ISG vs. INF
[Liga Latinoamérica] R7 vs. XTN
[Liga Latinoamérica] XTN vs. INF
[Liga Latinoamérica] ISG vs. XTN
[Liga Latinoamérica] FG vs. ISG
[Liga Latinoamérica] INF vs. PIX
[Liga Latinoamérica] INF vs. ISG
[Liga Latinoamérica] AK vs. R7


In [12]:
sum = 0
for key in league_calendar:
    sum += len(league_calendar[key].events)
print("Found", sum, "Events in Calendars")


Found 10638 Events in Calendars


In [13]:
# Save all calendars in individual files
for key in league_calendar:
    with open(output_dir + "/" + key + ".ics", 'w', encoding="utf-8") as file:
        file.writelines(league_calendar[key])

In [14]:
# write a ics file all events
c = generate_empty_calender("League of Legends Profispiele")
for key in league_calendar:
    for event in league_calendar[key].events:
        c.events.append(event)
with open(output_dir + "/" + "alle.ics", 'w', encoding="utf-8") as file:
    file.writelines(c)